In [ ]:
# packages and dependencies import
import pandas as pd
import seaborn as sns # Used for Exploratory Data Analysis
import numpy as np
import matplotlib
from pandas import DataFrame

# matplotlib.use('TkAgg')  # or 'QtAgg' if PyQt is installed {pip install PyQt6}
import matplotlib.pyplot as plt

In [ ]:
#Data Import 
data: DataFrame = pd.read_csv("/home/tjselevani/PycharmProjects/PythonProject/data/last-3-months-transactions.csv")

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data.shape

In [ ]:
data.columns

In [ ]:
data.describe()

In [ ]:
data.isnull().sum()

In [ ]:
#Data Visualization

In [ ]:
# Global variables for filtering
# Example: Select multiple vehicles (or set to None for all vehicles)
VEHICLE_BOOKED = "SM192"  # or None for all
TRANSACTION_TYPE = "CREDIT"

In [ ]:
# Filter data for the specific vehicle "sm191"
if VEHICLE_BOOKED is None:
    vehicle_data = data[data['transaction_type'] == TRANSACTION_TYPE].copy()
else:
    vehicle_data = data[(data['vehicle_booked'] == VEHICLE_BOOKED) & 
                        (data['transaction_type'] == TRANSACTION_TYPE)].copy()

In [ ]:
# Convert 'created_at' column to datetime if not already
vehicle_data['created_at'] = pd.to_datetime(vehicle_data['created_at'])

# Extract day, month, and week within the month
vehicle_data['day'] = vehicle_data['created_at'].dt.date
vehicle_data['month'] = vehicle_data['created_at'].dt.to_period('M').astype(str)
vehicle_data['week'] = vehicle_data['created_at'].dt.isocalendar().week

# Aggregate total fares by day and week
daily_fares = vehicle_data.groupby('day')['amount'].sum().reset_index()
weekly_fares = vehicle_data.groupby('week')['amount'].sum().reset_index()

# Line graph for daily progression
plt.figure(figsize=(12, 6))
sns.lineplot(x='day', y='amount', data=daily_fares, marker='o', label="Daily Fare")

plt.title(f"Daily Fare Progression for {VEHICLE_BOOKED}")
plt.xlabel("Day")
plt.ylabel("Total Fare")
plt.xticks(rotation=45)
plt.grid(True)

plt.tight_layout()  # Adjust layout so annotations fit

plt.show()

# Bar graph for weekly totals
plt.figure(figsize=(12, 6))
sns.barplot(x='week', y='amount', data=weekly_fares, palette="coolwarm")

plt.title(f"Weekly Total Fare for {VEHICLE_BOOKED}")
plt.xlabel("Week")
plt.ylabel("Total Fare")
plt.xticks(rotation=45)
plt.grid(axis='y')

# Add annotations for each bar
for i, amount in enumerate(weekly_fares['amount']):
    plt.text(i, amount, f"{amount}", ha='center', va='bottom')

plt.tight_layout()  # Adjust layout so annotations fit

plt.show()
